In [1]:
# summaflow library
from summaflow import (
    Stats,
    GeoLayer,
    SUMMAWorkflow,
)

# 3rd party libraries
import pandas as pd
import numpy as np
import geopandas as gpd

# built-in libraries
import os

In [2]:
# paths
root_path_layers = '/Users/kasrakeshavarz/Documents/github-repos/summa-model-specific/tests/notebooks/test-stats/'

# layers' paths
landcover_path = os.path.join(root_path_layers, 'landsat-landcover')
soilclass_path = os.path.join(root_path_layers, 'usda-soil')
merithdyr_path = os.path.join(root_path_layers, 'merit-hydro')

# geolayer's path
root_path_geoms = '../../../examples/bow-at-calgary-geofabric/'
riv_path = os.path.join(root_path_geoms, 'bcalgary_rivers.shp')
cat_path = os.path.join(root_path_geoms, 'bcalgary_subbasins.shp')
hru_path = os.path.join(root_path_geoms, 'bcalgary_subbasins.shp')

____

## Building GeoLayers

In [3]:
# layers needed by the setup workflow
# elevation
elv = GeoLayer.from_maf(
    maf_stats=os.path.join(merithdyr_path, 'summaflow_stats_elv.csv'),
    maf_layer=os.path.join(merithdyr_path, 'summaflow_elv.tif'),
    maf_geolayer=os.path.join(cat_path),
    unit = 'meters',
)
# landcover
landcover = GeoLayer.from_maf(
    maf_stats=os.path.join(landcover_path, 'summaflow_stats_NA_NALCMS_landcover_2020_30m.csv'),
    maf_layer=os.path.join(landcover_path, 'summaflow_NA_NALCMS_landcover_2020_30m.tif'),
    maf_geolayer=os.path.join(cat_path),
    unit = 'dimensionless',
)
# USDA soil classes
soil = GeoLayer.from_maf(
    maf_stats=os.path.join(soilclass_path, 'summaflow_stats_soil_classes.csv'),
    maf_layer=os.path.join(soilclass_path, 'summaflow_soil_classes.tif'),
    maf_geolayer=os.path.join(cat_path),
    unit = 'dimensionless',
)

In [4]:
elv

Stats: {'mean'}
Layer: True
Geometry: True
Geolayer Unit: meter

In [5]:
landcover

Stats: {'variety', 'frac', 'majority', 'max', 'min', 'mean', 'count', 'q', 'median', 'coefficient_of_variation', 'variance', 'minority', 'stdev'}
Layer: True
Geometry: True
Geolayer Unit: dimensionless

In [6]:
soil

Stats: {'frac', 'majority', 'max', 'min', 'mean', 'median', 'minority'}
Layer: True
Geometry: True
Geolayer Unit: dimensionless

____

## Reading Geometries

In [7]:
riv_obj = gpd.read_file(riv_path)
cat_obj = gpd.read_file(cat_path)
hru_obj = gpd.read_file(hru_path)

____

In [8]:
exp = SUMMAWorkflow(
    forcing_data = ['1', '2'],
    forcing_name_mapping = {
        'pr': 'precipitation',
        'temp': 'airtemp',
    },
    forcing_unit_mapping = {
        'pr': 'mm',
        'temp': 'degC',
    },
    forcing_to_unit_mapping = {
        'pr': 'kgm^/s',
        'temp': 'K',
    },
    forcing_attrs = {
        'measurement_height': 40,
        'measurement_height_unit': 'meter',
        'local': {
            'pr': {
                'long_name': 'precipitation at the surface',
            },
            'temp': {
                'long_name': 'Air temperature',
            },
        },
        'global': {},
    },
    topology_data = {
        'riv': riv_obj,
        'hru': hru_obj,
        'cat': cat_obj,
    },
    topology_unit_mapping = {'a': 'm', 'c': 'ha'},
    topology_to_unit_mapping = {'a': 'bar', 'c': 'dca'},
    topology_attrs = {
        'gru_fid': 'COMID',
        'hru_fid': 'COMID',
    },
    geospatial_data= {
        'elevation': elv,
        'soil': soil,
        'landcover': landcover,
    },
)

In [9]:
exp.topology_attrs

{'gru_fid': 'COMID', 'hru_fid': 'COMID'}

In [10]:
exp.init_attrs()

/Users/kasrakeshavarz/Documents/github-repos/summa-model-specific/src/summaflow/utils.py:197: UserWarning: Using EPSG:3857 for centroid calculations on geographic data. For better accuracy, specify a local projection using calculation_crs parameter.
  warnings.warn(


In [11]:
exp.attrs

<xarray.Dataset> Size: 18kB
Dimensions:         (gru: 169, hru: 169)
Coordinates:
  * gru             (gru) int64 1kB 71027942 71027957 ... 71039250 71039266
  * hru             (hru) int64 1kB 71027942 71027957 ... 71039250 71039266
Data variables:
    mHeight         (hru) int64 1kB 40 40 40 40 40 40 40 ... 40 40 40 40 40 40
    slopeTypeIndex  (hru) int64 1kB 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1
    gruId           (gru) int64 1kB 71027942 71027957 ... 71039250 71039266
    hruId           (hru) int64 1kB 71027942 71027957 ... 71039250 71039266
    hru2gruId       (hru) int64 1kB 71027942 71027957 ... 71039250 71039266
    latitude        (hru) float64 1kB -114.2 -114.4 -114.5 ... -115.0 -115.3
    longitude       (hru) float64 1kB 51.1 51.16 51.18 ... 50.92 50.95 50.94
    HRUarea         (hru) float64 1kB 1.528e+08 9.333e+07 ... 1.135e+08
    elevation       (hru) float64 1kB 1.16e+03 1.204e+03 ... 1.913e+03 2.121e+03
    vegTypeIndex    (hru) int64 1kB 17 10 17 17 8 8 8 8 ... 16 16 16 16 16 1 16
    soilTypeIndex   (hru) int64 1kB 3 3 3 3 3 3 3 3 3 3 ... 3 3 3 3 3 3 3 3 3 3